In [17]:
# Dependencies
import requests
import json
import pandas as pd
import time

# Import the API key
from config import api_key

In [18]:
# Variables to build the endpoint URL
begin_date = "01-2000"
end_date = "12-2023"
time_frame = f'?from={begin_date}&to={end_date}'
crime = ["homicide", "rape", "robbery", "aggravated-assault", "arson", "burglary", "larceny", "motor-vehicle-theft"]

# Loop to pull json data for each crime, sort it, and write it to a csv file
for c in crime:

    # Set the variable lists to clean them at the start of a new loop
    months = []
    total_crime = []
    total_clearance = []
    crime_per_capita = []
    clear_per_capita = []
    total_pop = []
    participated_pop = []
    pop_coverage = []
    perc_pros = []
    
    # Setting the base URL based on above variables
    base_url = f'https://api.usa.gov/crime/fbi/cde/summarized/national/{c}{time_frame}{api_key}'
    
    # Request the data from FBI CDE
    crime_data = requests.get(base_url).json()
    
    # Put the returned data in to a DataFrame
    df = pd.DataFrame(crime_data)
    
    # Run through the DataFrame for variable crime numbers per month and append to total_crime
    for index in df["offenses"]["actuals"]["United States"]:
        total_crime.append(df["offenses"]["actuals"]["United States"][index])
        
        # Grab the Month (index) and append it to the months list to be our index in final DataFrame
        months.append(index)
    
    # Run through the DataFrame for Total Prosecutions of variable crime for each month and append to total_clearance
    for index in df["offenses"]["actuals"]["United States Clearances"]:
        total_clearance.append(df["offenses"]["actuals"]["United States Clearances"][index])
    
    # Run through the DataFrame for Crime Per Capita for variable crime each month and append to crime_per_capita
    for index in df["offenses"]["rates"]["United States"]:
        crime_per_capita.append(df["offenses"]["rates"]["United States"][index])
    
    # Run through the DataFrame for Crime Prosecuted Per Capita for variable crime each month and append to clear_per_capita
    for index in df["offenses"]["rates"]["United States Clearances"]:
        clear_per_capita.append(df["offenses"]["rates"]["United States Clearances"][index])

    # Run through the DataFrame for Total Population for each month and append to total_pop  
    for index in df["populations"]["population"]["United States"]:
        total_pop.append(df["populations"]["population"]["United States"][index])

    # Run through the DataFrame for Participated Population for each month and append to participated_pop    
    for index in df["populations"]["participated_population"]["United States"]:
        participated_pop.append(df["populations"]["participated_population"]["United States"][index])

    # Run through the DataFrame for Percentage of Population Covered by the Data and append to pop_coverage    
    for index in df["tooltips"]["Percent of Population Coverage"]["United States"]:
        pop_coverage.append(df["tooltips"]["Percent of Population Coverage"]["United States"][index])

    # Creating the DataFrame to store data gathered above starting with the Index(months) and first Column Total Crimes
    df_crime_data = pd.DataFrame(total_crime, index=months, columns=[f"Total {c}"])
    
    df_crime_data["Clearances"] = total_clearance # Adding Column for Total Prosecutions
    df_crime_data["Crime(Per 100k)"] = crime_per_capita # Adding Column for Crime per 100k
    df_crime_data["Prosecutions(Per 100k)"] = clear_per_capita # Adding Column for Clearance per 100k
    df_crime_data["Prosecuted Perc"] = (df_crime_data["Clearances"] / df_crime_data[f"Total {c}"]).round(2) # Adding Column for Prosecution Percentage, and calculating, and rounding.
    df_crime_data["Total Pop"] = total_pop # Adding Column for Total Population
    df_crime_data["Partic Pop"] = participated_pop # Adding Column for Participated Population
    df_crime_data["Pop Coverage"] = pop_coverage # Adding Column for Population Coverage of Data
    
    # Converting the Index(Months) to Date/Time format
    df_crime_data.index = pd.to_datetime(df_crime_data.index, format='%m-%Y', errors='coerce')
    
    # Creating a new DataFrame with the Index Sorted from Earliest to Latest Dates
    df_sorted = df_crime_data.sort_index()
    
    # Writing the sorted DataFrame to a csv file
    df_sorted.to_csv(f"../Resources/National/{c}_national_data.csv")
    
    # Pause for 2 seconds as to not overload the API.
    time.sleep(2)
